In [16]:
import pandas as pd
import os

In [6]:
promising = pd.read_csv('/Users/caged/Dropbox/RSN Mining/promising indicators.csv')
promising.head()

,orig_id,description,new_id
0,1b,Specifically note that suppliers may continue ...,MA1_1A_2
1,3C,Indicate which 3TG minerals are used,MA2_3_3
2,7A,"Sort countries of origin by mineral, product, ...",MA2_6_1
3,14A,Conducted Independent Private Sector Audit,MA4_13_1
4,19B,Source from an in-region initiative not just i...,MA5_18_2


In [21]:
ind_ma1 = pd.read_csv('/Users/caged/Dropbox/RSN Mining/indicator_data_matched_with_cik_data.csv', dtype='str', usecols=['MA1_1A_2', 'cik', 'primarysymbol'])
ind_ma1 = ind_ma1.rename(columns={'MA1_1A_2': 'label'})
ind_ma1['dir_name'] = ind_ma1.apply(lambda row: ' '.join([row.primarysymbol, '--', row.cik]), axis=1)
ind_ma1['dir_path'] = ind_ma1.dir_name.apply(lambda x: os.path.join('supporting_docs_2015', x))
ind_ma1.head()

,label,cik,primarysymbol,dir_name,dir_path
0,1,66740,MMM,MMM -- 66740,supporting_docs_2015/MMM -- 66740
1,1,937966,ASML,ASML -- 937966,supporting_docs_2015/ASML -- 937966
2,0,1800,ABT,ABT -- 1800,supporting_docs_2015/ABT -- 1800
3,0,1144215,AYI,AYI -- 1144215,supporting_docs_2015/AYI -- 1144215
4,1,886125,ALU,ALU -- 886125,supporting_docs_2015/ALU -- 886125


In [28]:
# How to walk a folder
row_1 = ind_ma1.ix[0]
for file_name in os.listdir(row_1.dir_path):
    file_path = os.path.join(row_1.dir_path, file_name)
    print(file_path)

supporting_docs_2015/MMM -- 66740/0001104659-15-042396.txt
supporting_docs_2015/MMM -- 66740/a15-12384_1ex1d01.htm
supporting_docs_2015/MMM -- 66740/a15-12384_1sd.htm


In [37]:
dd = pd.read_csv('disclosure_docs_list.csv', dtype='str')
dd = dd.drop('Unnamed: 0', axis=1)
dd = dd.rename(columns={'req_cik': 'req_company_name', 'req_company_name': 'req_cik'})
dd.ret_filing_date = pd.to_datetime(dd.ret_filing_date)
dd_2015 = dd[dd.ret_filing_date.dt.year == 2015]
dd_2015.head()

,req_company_name,req_cik,req_url,ret_cik,ret_company_name,ret_description,ret_filing_date,ret_sec_accession_number,ret_title,ret_url
2,AARON'S INC,706688,http://www.sec.gov/Archives/edgar/data/706688/...,0000706688,AARON'S INC,sdconflictminerals2014.htm,2015-05-22 12:06:05,0000706688-15-000148,SD_CONFLICT_MINERALS_2014,https://www.sec.gov/Archives/edgar/data/706688...
3,AARON'S INC,706688,http://www.sec.gov/Archives/edgar/data/706688/...,0000706688,AARON'S INC,0000706688-15-000148.txt,2015-05-22 12:06:05,0000706688-15-000148,Complete submission text file,https://www.sec.gov/Archives/edgar/data/706688...
7,AAON INC,824142,http://www.sec.gov/Archives/edgar/data/824142/...,0000824142,AAON INC,a2014formsd.htm,2015-05-29 11:36:18,0000824142-15-000071,SD,https://www.sec.gov/Archives/edgar/data/824142...
8,AAON INC,824142,http://www.sec.gov/Archives/edgar/data/824142/...,0000824142,AAON INC,exhibit101.htm,2015-05-29 11:36:18,0000824142-15-000071,EXHIBIT 1.01,https://www.sec.gov/Archives/edgar/data/824142...
9,AAON INC,824142,http://www.sec.gov/Archives/edgar/data/824142/...,0000824142,AAON INC,0000824142-15-000071.txt,2015-05-29 11:36:18,0000824142-15-000071,Complete submission text file,https://www.sec.gov/Archives/edgar/data/824142...


In [38]:
row_1 = ind_ma1.ix[0]

In [41]:
# Get just the text file
dd_2015[(dd_2015.req_cik == row_1.cik) & (dd_2015.ret_title == 'Complete submission text file')]

,req_company_name,req_cik,req_url,ret_cik,ret_company_name,ret_description,ret_filing_date,ret_sec_accession_number,ret_title,ret_url
50,3M CO,66740,http://www.sec.gov/Archives/edgar/data/66740/0...,0000066740,3M CO,0001104659-15-042396.txt,2015-06-01 15:06:30,0001104659-15-042396,Complete submission text file,https://www.sec.gov/Archives/edgar/data/66740/...


In [56]:
results = dd_2015[(dd_2015.req_cik == row_1.cik) & (dd_2015.ret_title == 'Complete submission text file')].reset_index()
assert len(results) == 1
text_file_name = results['ret_description'][0]
text_file_name

'0001104659-15-042396.txt'

In [60]:
file_path = os.path.join(row_1.dir_path, text_file_name)
with open(file_path, 'r') as f:
    text = f.read()

In [62]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(text)

In [73]:
documents = soup.findAll('document')
documents[0].text

'\nSD\n1\na15-12384_1sd.htm\nSD\n\n\n\n\n\n\n\xa0\n\n\xa0\n\n\xa0\nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWASHINGTON, DC 20549\n\xa0\nFORM\xa0SD\n\xa0\nSpecialized Disclosure Report\n\xa0\n3M COMPANY\n(Exact Name of Registrant as Specified in Its Charter)\n\xa0\nDelaware\n(State or Other Jurisdiction of Incorporation)\n\xa0\n\n\n\n\nFile No.\xa01-3285 \n\n\xa0 \n\n41-0417775  \n\n\n(Commission File Number) \n\n\xa0 \n\n(IRS Employer Identification No.)  \n\n\n\xa0 \n\n\xa0 \n\n\xa0  \n\n\n3M Center, St. Paul,   Minnesota \n\n\xa0 \n\n55144-1000  \n\n\n(Address of Principal Executive   Offices) \n\n\xa0 \n\n(Zip Code)   \n\n\xa0\nMark Copman, Vice President\n3M Sourcing Operations\n(651) 733-1110\n(Name and telephone number, including area code, of the person to contact in connection with this report)\n\xa0\nCheck the appropriate box to indicate the rule\xa0pursuant to which this form is being filed, and provide the period to which the information in this form applies:\n\xa0